In [1]:
import os
import re
import pandas as pd
from collections import defaultdict
import sqlite3


In [2]:
conn = sqlite3.connect('data/yw_model.db')

### Functions for parsing the yw models file. 

In [3]:
### function extracting the programs.

def extract_steps(line):
    
    #% FACT: program(program_id, program_name, qualified_program_name, begin_annotation_id, end_annotation_id).
    #print("Extracting steps.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    
    data = data.split(',')
    #print(data)
    
    
    ins_query = [data[0].strip().strip("'"),data[1].strip().strip("'"),data[2].strip().strip("'")]
    conn.execute("""insert into steps values(?,?,?)""", ins_query)
    conn.commit()
    return

In [4]:
### function extracting the workflow.

def extract_workflows(line):
    
    #print(line)
    data = line[line.index("(")+1:line.index(")")]
    #print(data)
    conn.execute("""insert into workflow values(?)""", data[0].strip().strip("'"))
    conn.commit()
      
    return 

In [5]:
### function extracting the ports

def extract_ports(line):
    #print("Extracting ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    
    #% FACT: port(port_id, port_type, port_name, qualified_port_name, port_annotation_id, data_id).
    
    data = data.split(',')
    
    if data[1].strip().strip("'").upper() == 'PARAM':
        port_type = 'IN'
    else:
        port_type = data[1].strip().strip("'").upper()
    
    ins_query = [data[0].strip().strip("'"),port_type,data[2].strip().strip("'"),data[3].strip().strip("'")
                 ,data[4].strip().strip("'"),data[5].strip().strip("'")]
    conn.execute("""insert into port values(?,?,?,?,?,?)""", ins_query)
    conn.commit()
    return 

In [6]:
### function extracting the input ports

def input_ports(line):
    #print("Extracting Input ports.")
    
    ## Extract data between "(" and ")".
    ## % FACT: has_in_port(step_id, port_id).

    data = line[line.index("(")+1:line.index(")")]
    
    data = data.split(',')
    ins_query = [data[0].strip().strip("'"),data[1].strip().strip("'")]
    conn.execute("""insert into has_in_port values(?,?)""", ins_query)
    conn.commit()
    
    #in_ports[data[0]].append((data[0].strip(), data[1].strip()))
    
    return

In [7]:
### function extracting the output ports 

def output_ports(line): 
    #print("Extracting output ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    ins_query = [data[0].strip().strip("'"),data[1].strip().strip("'")]
    conn.execute("""insert into has_out_port values(?,?)""", ins_query)
    conn.commit()
    #out_ports[data[0]].append((data[0].strip(), data[1].strip()))
    #print(data)
    return

In [8]:
### function extracting the subprograms 

def has_subprogram(line): 
    #print("Extracting output ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    
    ins_query = [data[0].strip().strip("'"),data[1].strip().strip("'")]
    conn.execute("""insert into has_subprogram values(?,?)""", ins_query)
    conn.commit()
    
    return

In [9]:
### function extracting the output ports 

def get_port_connects_to_channel(line): 
     
    data = line[line.index("(")+1 : line.index(")")]
    data = data.split(',')

    ins_query = [data[0].strip().strip("'"),data[1].strip().strip("'")]
    conn.execute("""insert into port_connects_to_channel values(?,?)""", ins_query)
    conn.commit()
    
    return

In [10]:
def get_inflow_connections(line): 
     
    #print(line)
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    #inflow_conn[data[0]]= data[1].strip()
    
    ins_query = [data[0].strip().strip("'"),data[1].strip().strip("'")]
    conn.execute("""insert into inflow_connects_to_channel values(?,?)""", ins_query)
    conn.commit()
    
    return

In [11]:
def get_outflow_connections(line): 
    #print("Extracting output ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    #outflow_conn[data[0]]= data[1].strip()

    ins_query = [data[0].strip().strip("'"),data[1].strip().strip("'")]
    conn.execute("""insert into outflow_connects_to_channel values(?,?)""", ins_query)
    conn.commit()
    return 

In [12]:
### function extracting the output ports 

def get_port_data(line): 
    #print("Extracting output ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    port_data[data[0]]= { 
                          "data_name" : data[1].strip().strip("'"), 
                          "qual_data_name" : data[2].strip()
                        } 
    #print(data)
    return

In [13]:
### function extracting the output ports 

def get_channel(line): 
    #print("Extracting output ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    
    ins_query = [data[0].strip().strip("'"),data[1].strip().strip("'")]
    conn.execute("""insert into channel values(?,?)""", ins_query)
    conn.commit()
    #channel[data[0]]= data[1].strip()
    #print(data)
    return

In [14]:
### function extracting the port alias

def port_alias(line):
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    
    port_id = data[0].strip()
    alias = data[1].strip().strip("'")
    
    #print(port_name, port_id,qualified_name)
    
    ins_query = [data[0].strip().strip("'"),data[1].strip().strip("'")]
    conn.execute("""insert into port_alias values(?,?)""", ins_query)
    conn.commit()
    
    '''
    ### regex for splitting the string and getting qualified program name
    regex = re.compile(r'>|<')
    qname = regex.split(qualified_name)
    
    pname = qname[0].split('.')[-1] 
    
    if pname.find('-') > -1:
        port_alt_name[alias] = pname[:-1]+ '/' +port_name
    else:
        port_alt_name[alias] = pname + '/' +port_name
       
    '''
    return 



In [15]:
def read_yw_model(filename):
    regex = re.compile(r'^%')
    chk_channel = re.compile(r'^ch')
    with open(filename, "r") as yw_struct:
        line = yw_struct.readline()
        while line:
            #print(line)
            comments = regex.match(line)
            if comments: 
                #print(line)
                line = yw_struct.readline()
            else:  
                if("program(" in line and "has_subprogram" not in line):
                    extract_steps(line)
                elif("workflow(" in line):
                    extract_workflows(line)            
                elif("has_subprogram(" in line):
                    has_subprogram(line)            
                elif("port(" in line  and "has_in_port(" not in line and "has_out_port" not in line):
                #    print(line)
                    extract_ports(line)
                elif("has_in_port(" in line):
                    input_ports(line)
                elif("has_out_port(" in line ):
                    output_ports(line)
                elif ("port_alias(" in line ): 
                    port_alias(line)
                elif("data(" in line):
                    get_port_data(line)
                elif("port_connects_to_channel(" in line):
                    #print(line)
                    get_port_connects_to_channel(line)
                elif("inflow_connects_to_channel("in line):
                    #print(line)
                    get_inflow_connections(line)                    
                elif("outflow_connects_to_channel("in line):
                    get_outflow_connections(line)
                elif( chk_channel.match(line) ):
                    get_channel(line)

                line = yw_struct.readline()
        
        #get_in_out_ports()

In [66]:

conn.execute (''' drop table channel ''')                    
conn.execute (''' drop table data  ''')                 
conn.execute (''' drop table has_in_port ''')                  
conn.execute (''' drop table has_out_port ''')                
conn.execute (''' drop table has_subprogram ''')              
conn.execute (''' drop table inflow_connects_to_channel ''')  
conn.execute (''' drop table outflow_connects_to_channel ''')
conn.execute (''' drop table port ''')
conn.execute (''' drop table port_alias ''')
conn.execute (''' drop table port_connects_to_channel ''')
conn.execute (''' drop table steps ''')
conn.execute (''' drop table workflow ''')
conn.execute (''' drop table cwl_steps_info ''')
conn.execute (''' drop table wf_ports ''')
conn.execute (''' drop table qual_portname ''')

In [67]:
## Create tables for storing the data model. 



# % FACT: program(program_id, program_name, qualified_program_name, begin_annotation_id, end_annotation_id).
conn.execute("""create table steps (
                program_id      int     primary key not NULL ,
                program_name      text,
                qualified_program_name  text)
            """)

#% FACT: workflow(program_id).

conn.execute("""create table workflow (
                program_id      int     primary key not NULL
                )
            """)

#% FACT: has_subprogram(program_id, subprogram_id).

conn.execute("""create table has_subprogram (
                program_id  int,
                sub_program_id      int
                )
            """)

#% FACT: port(port_id, port_type, port_name, qualified_port_name, port_annotation_id, data_id).


conn.execute("""create table port (
                port_id      int     primary key not NULL,
                port_type  int,
                port_name  text,
                qualified_port_name text,
                port_annotation_id int, 
                data_id int 
                )
            """)



# % FACT: port_alias(port_id, alias).

conn.execute("""create table port_alias (
                port_id    int  ,
                alias    text
                )
            """)

#% FACT: has_in_port(block_id, port_id).

conn.execute("""create table has_in_port (
                block_id int,
                port_id    int
                )
            """)

#% FACT: has_out_port(block_id, port_id).

conn.execute("""create table has_out_port (
                block_id int,
                port_id    int
                )
            """)

#% FACT: data(data_id, data_name, qualified_data_name).

conn.execute("""create table data (
                data_id int,
                data_name    text,
                qualified_data_name text
                )
            """)

#% FACT: channel(channel_id, data_id).


conn.execute("""create table channel (
                channel_id int,
                data_id    int
                )
            """)

#% FACT: port_connects_to_channel(port_id, channel_id).

conn.execute("""create table port_connects_to_channel (
                port_id int,
                channel_id    int
                )
            """)
    
# % FACT: inflow_connects_to_channel(port_id, channel_id).
conn.execute("""create table inflow_connects_to_channel (
                port_id int,
                channel_id    int
                )
            """)


#% FACT: outflow_connects_to_channel(port_id, channel_id).
conn.execute("""create table outflow_connects_to_channel (
                port_id int,
                channel_id    int
                )
            """)

In [68]:
port_data = {}
yw_model_file = "example_sql/paleocar_models/paleocar_models.P"

read_yw_model(yw_model_file)


In [69]:
sql_steps_in_out_ports = '''
select 
	wf.wf_id, s.program_id, s.program_name, p.port_name, p.port_id, p.port_type, p.data_id  
from 
	steps s, port p, has_in_port inp, 
	(select 
		sub_program_id  , program_id as wf_id 
	 from 
		has_subprogram
	) wf
where 
	s.program_id = inp.block_id
and	p.port_id = inp.port_id 
and inp.block_id  = wf.sub_program_id
union 
select 
	wf.wf_id, s.program_id, s.program_name, p.port_name,p.port_id, p.port_type , p.data_id
from 
	steps s, port p, has_out_port out,
	(select 
		sub_program_id  , program_id as wf_id 
	 from 
		has_subprogram
	) wf
where 
	s.program_id = out.block_id
and	p.port_id = out.port_id 
and out.block_id = wf.sub_program_id
'''


sql_wf_ports = '''
select 
	s.program_id, s.program_name, p.port_name, p.port_id, p.port_type, p.data_id  
from 
	steps s, port p, has_in_port inp 
where 
	s.program_id = inp.block_id
and	p.port_id = inp.port_id 
and inp.block_id in 
	(select 
		program_id  
	 from 
		workflow
	)
union 
select 
	s.program_id, s.program_name, p.port_name,p.port_id, p.port_type , p.data_id
from 
	steps s, port p, has_out_port out 
where 
	s.program_id = out.block_id
and	p.port_id = out.port_id 
and out.block_id in 
	(select 
		program_id  
	 from 
		workflow
	)
'''

sql_qual_portname = '''
select 
	p.program_name, p.port_name , q.port_name as qualified_portname, p.port_type , p.data_id
from
	cwl_steps_info p, wf_ports q
 where upper(p.port_type) in ('IN', 'PARAM')
 and p.port_name = q.port_name
 union
select 
	p.program_name, p.port_name , q.port_name as qualified_portname, p.port_type, p.data_id
from
	cwl_steps_info p, wf_ports q, port_alias pa
 where upper(p.port_type) in ('IN', 'PARAM') 
 and p.port_id = pa.port_id
 and pa.alias = q.port_name
union 
select
	p.program_name, p.port_name, p.program_name || '/' || p.port_name as qualified_portname, p.port_type , p.data_id
from
	cwl_steps_info p
where upper(p.port_type)= 'OUT'

'''

sql_wf_qual_out_port = '''
select 
	p.program_name, q.port_name, p.program_name || '/' || p.port_name as qualified_portname, p.port_type , p.data_id
from 
	cwl_steps_info p , port q
where q.port_id = p.port_id
and p.port_id in 
	(
	select 
		port_id
	from 
		port_connects_to_channel
	where 
		channel_id in 
		(select 
			max(channel_id)
		from 
			outflow_connects_to_channel
		group by port_id
		)
	)
union 
select 
	p.program_name, pa.alias as port_name, p.program_name || '/' || p.port_name as qualified_portname, p.port_type , p.data_id
from 
	cwl_steps_info p , port q, port_alias pa
where q.port_id = p.port_id
and p.port_id = pa.port_id
and p.port_id in 
	(
	select 
		port_id
	from 
		port_connects_to_channel
	where 
		channel_id in 
		(select 
			max(channel_id)
		from 
			outflow_connects_to_channel
		group by port_id
		)
	)
union
select 
	p.qualified_port_name, p.port_name,  p.qualified_port_name, p.port_type , p.data_id
from 
	port p 
where p.port_name like '_YW_%'
'''

sql_dangling_port_id = """
select 
	port_id 
from
	port 
where data_id in 
(
	select distinct data_id from port  
	EXCEPT
	select distinct data_id  from channel
)
"""

#df_dangling_ports = pd.read_sql_query(sql_dangling_port_id, con=conn)



In [70]:
conn.execute('create table cwl_steps_info as ' + sql_steps_in_out_ports)
conn.execute('create table wf_ports as ' + sql_wf_ports)
conn.execute('create table qual_portname as ' + sql_qual_portname)

In [71]:
cwl_file_df = pd.DataFrame()
wf_port_df = pd.DataFrame()
qual_portname = pd.DataFrame()
qual_wf_out_port = pd.DataFrame()
df_dangling_ports = pd.DataFrame()

In [72]:
cwl_file_df = pd.read_sql_query(sql_steps_in_out_ports, con=conn)
wf_port_df = pd.read_sql_query(sql_wf_ports, con=conn)
qual_portname = pd.read_sql_query(sql_qual_portname, con=conn)
qual_wf_out_port = pd.read_sql_query(sql_wf_qual_out_port, con=conn)
df_dangling_ports = pd.read_sql_query(sql_dangling_port_id, con=conn)

In [73]:
## Add an output port in the ports tables
def ins_out_port(program_name):
    wf_id = cwl_file_df[cwl_file_df['program_name'] == program_name]['wf_id'].values[0]
    prog_id = cwl_file_df[cwl_file_df['program_name'] == program_name]['program_id'].values[0]
    port_id = cwl_file_df['port_id'].max() + 1
    data_id = cwl_file_df['data_id'].max() + 1
    #print(port_id, data_id)
    port_type = 'OUT'
    port_name = '_YW_OUT_'+ program_name
    qual_prog_name = program_name + '/' + port_name
    sql = """
    insert into port values (?,?,?,?,?,?)
    """
    data = [int(port_id), port_type, port_name,qual_prog_name,port_id,int(data_id)]
    conn.execute(sql,data)
    
    sql = """
    insert into has_out_port values (?,?)
    """
    data = [int(wf_id),int(port_id)]
    conn.execute(sql,data)
    data = [int(prog_id),int(port_id)]
    conn.execute(sql,data)
    conn.commit()

In [74]:
## Add an output port in the ports tables
def ins_link_wf_port(port_id, wf_id,port_type,program_name):
    #print(port_id, data_id)
    new_port_id = get_max_port_id() + 1
    data_id = cwl_file_df[cwl_file_df['port_id'] == port_id]['data_id']
    #print(type(new_port_id))
    
    port_name = cwl_file_df[cwl_file_df['port_id'] == port_id]['port_name'].values[0]
    qual_prog_name = program_name + '/' + port_name
    print(port_name)
    if port_type == 'OUT':
        wf_port_name = '_YW_OUT_'+ port_name
        sql_link = """
                   insert into has_out_port values (?,?)
                   """
        values = [int(wf_id),int(new_port_id)]
    else: 
        wf_port_name = '_YW_IN_'+ port_name
        sql_link = """
                   insert into has_in_port values (?,?)
                   """   
        values = [int(wf_id),int(new_port_id)]
        
        
    sql = """
    insert into port values (?,?,?,?,?,?)
    """
    
    data = [int(new_port_id), port_type, wf_port_name,qual_prog_name,port_id,int(data_id)]
    conn.execute(sql,data)
    #print(data)
    #print(values)
    conn.execute(sql_link,values)
    conn.commit()
    
    #rebuild_dataframes()

In [75]:
def get_max_port_id():
    cur = conn.cursor()
    cur.execute('''SELECT max(port_id) from port ''')
    max_port_id = cur.fetchone()
    
    return int(max_port_id[0])

## Check for the dangling ports and create 
## the respective ports 

In [76]:
## Get the list of the dangling port id's and pass into the DF. 
for idx in cwl_file_df[cwl_file_df['port_id'].isin(df_dangling_ports['port_id'])].index:
    wf_id = cwl_file_df.loc[idx]['wf_id']
    port_id = cwl_file_df.loc[idx]['port_id']
    port_type = cwl_file_df.loc[idx]['port_type']
    program_name = cwl_file_df.loc[idx]['program_name']
    ins_link_wf_port(port_id, wf_id,port_type,program_name)

In [77]:
### Check for the programs without output ports
### use the ins_out_port to create the output port for wf and prog. 

program_name_uniq = cwl_file_df['program_name'].unique()
program_with_out  = cwl_file_df[cwl_file_df['port_type'] == 'OUT']["program_name"].unique()
prog_wo_outport = list(set(program_name_uniq).difference(set(program_with_out)))

for prog in prog_wo_outport:
    #print(prog)
    ins_out_port(prog)



In [78]:
### Reload the dataframes now 
cwl_file_df = pd.read_sql_query(sql_steps_in_out_ports, con=conn)
wf_port_df = pd.read_sql_query(sql_wf_ports, con=conn)
qual_portname = pd.read_sql_query(sql_qual_portname, con=conn)
qual_wf_out_port = pd.read_sql_query(sql_wf_qual_out_port, con=conn)
df_dangling_ports = pd.read_sql_query(sql_dangling_port_id, con=conn)


In [79]:
def get_filename(program_id, is_wf):
    wfid_list = wf_port_df['program_id'].unique()
    if is_wf:
        program_name =wf_port_df[wf_port_df['program_id'] ==program_id]['program_name'].values[0]
        return 'wf_' + program_name + '.cwl'
    elif program_id in wfid_list:      
        program_name = cwl_file_df[cwl_file_df['program_id'] ==program_id]['program_name'].values[0]
        return 'wf' + program_name + '.cwl'
    else:
        program_name = cwl_file_df[cwl_file_df['program_id'] ==program_id]['program_name'].values[0]
        return program_name + '.cwl'

In [80]:
def write_cwl_file(filename, header, input_params, output_params):
    #print(filename)
    with open(filename,'w+') as writer: 
        writer.write(header)
        writer.write(input_params)
        writer.write(output_params)


In [125]:
def get_wf_steps(workflow_id):
    header_step = "steps: \n "
    buffer=''
    prog_list = cwl_file_df[cwl_file_df['wf_id'] == workflow_id]['program_id'].unique()
    
    for prog in prog_list: 
        outport_list= []
        prog_name = cwl_file_df[cwl_file_df['program_id'] ==prog]['program_name'].values[0]
        filename = get_filename(prog,False)
        
        buffer = buffer + prog_name + ': \n ' + ' run: ' +filename + ' \n ' + ' in: \n'
        
        for pid in cwl_file_df[cwl_file_df["program_id"]==prog].index:
            
            port_name = cwl_file_df.loc[pid]['port_name'] 
            port_type = cwl_file_df.loc[pid]['port_type']       
            data_id = cwl_file_df.loc[pid]['data_id']
            #print(port_name, data_id)
            
            if port_type == 'IN':
                #qname = qual_portname[qual_portname['port_name']== port_name]['qualified_portname'].values[0]
                filter_port = (qual_portname['port_name']== port_name) & (qual_portname['data_id']== data_id)& (qual_portname['port_type']== 'OUT')
                qn = qual_portname[filter_port]['qualified_portname']
                #print(qn)
                if qn.empty:
                    qname = port_name
                else:
                    qname = qn.values[0]
                #print(qname)
                buffer = buffer + '   ' + port_name + ': ' + qname + '\n'
            else: 
                outport_list.append(port_name)
        
        buffer = buffer + "  out: [" + ','.join(outport_list) + '] \n '    
    
    return header_step + buffer

In [126]:
t = qual_portname[qual_portname['port_name']=='max_preds']
t[(t['data_id'] == 13) & (t["port_type"]=='OUT')]

,program_name,port_name,qualified_portname,port_type,data_id
15,get_predictor_matrix,max_preds,get_predictor_matrix/max_preds,OUT,13


In [128]:
get_wf_steps(7)

'steps: \n defineLinearModels: \n  run: defineLinearModels.cwl \n  in: \n   carscores: carscores\n   max_preds: max_preds\n   predlist: predlist\n  out: [matches,models] \n calculateLinearModels: \n  run: calculateLinearModels.cwl \n  in: \n   matches: defineLinearModels/matches\n   models: defineLinearModels/models\n  out: [coefficients,model_errors] \n simplifyLinearModels: \n  run: simplifyLinearModels.cwl \n  in: \n   coefficients: calculateLinearModels/coefficients\n   model_errors: calculateLinearModels/model_errors\n  out: [final_models] \n '

In [129]:
## creating the cwl files and the wf.

In [130]:
"""
cur = conn.cursor()
cur.execute(''' select program_id from workflow order by 1 desc''')
"""

wf_id = cwl_file_df['wf_id'].unique()
prog_list = cwl_file_df['program_id'].unique()
dirname = 'example_sql/' + wf_port_df[wf_port_df["program_id"] ==1]['program_name'].values[0]



In [131]:
for prog_id in prog_list: 

    cwl_file_header = '''
#!/usr/bin/env cwl-runner
cwlVersion: v1.0
class: CommandLineTool
'''
    input_buffer = 'inputs: \n'
    output_buffer = 'outputs: \n'
    
    pname = cwl_file_df[cwl_file_df["program_id"] == [prog_id]]['program_name'].values[0]
    #print(prog_id)
    filename = dirname + '/' + get_filename(prog_id, False)

    #print(filename)
    for p_idx in cwl_file_df[cwl_file_df["program_id"] == [prog_id]].index:
        #pname = cwl_file_df.loc[p_idx]['program_name']
        pid = cwl_file_df.loc[p_idx]['program_id']
        #print(pname)
        port_name = cwl_file_df.loc[p_idx]['port_name'] 
        port_type = cwl_file_df.loc[p_idx]['port_type'] 
        #print(port_name, port_type)

        if port_type == 'IN':
            input_buffer = input_buffer + ' '+ port_name + ': \n'+ '  type: string \n \n'
            #print(input_buffer)
        else: 
            output_buffer = output_buffer + ' '+ port_name + ': \n'+ '  type: string \n \n'
            #print(output_buffer)
    
    output_buffer = output_buffer + "baseCommand: "
    write_cwl_file(filename, cwl_file_header, input_buffer, output_buffer)

In [132]:
cwl_wf_header = '''
#!/usr/bin/env cwl-runner
cwlVersion: v1.0
class: Workflow
requirements:
    - class: SubworkflowFeatureRequirement

'''
workflow_list = wf_port_df['program_id'].unique()

for id in sorted(workflow_list, reverse=True):
    input_buffer = 'inputs: \n'
    output_buffer = 'outputs: \n'
    #print(id)
    wf_name = wf_port_df[wf_port_df['program_id']== id]["program_name"].values[0]
    filename= dirname +'/' + get_filename(id, True)
    #print(filename)
    for p_id in wf_port_df[wf_port_df['program_id']== id].index:
        prog_id = wf_port_df.loc[p_id]['program_id']
        #print(int(prog_id))


        pname = wf_port_df.loc[p_id]['program_name']
        port_name = wf_port_df.loc[p_id]['port_name'] 
        port_type = wf_port_df.loc[p_id]['port_type'] 


        if port_type == 'IN':
            input_buffer = input_buffer + ' '+ port_name + ': \n'+ '  type: string \n \n'
            #print(input_buffer)
        else: 
            qname = qual_wf_out_port[qual_wf_out_port['port_name'] == port_name]['qualified_portname'].values[0]
            output_buffer = output_buffer + '  ' +  port_name  + ': \n' + '   type: string \n   outputSource: '+ qname +'\n'
            #print(output_buffer)
    
    wf_step_contents = get_wf_steps(id)
    write_cwl_file(filename,cwl_wf_header + input_buffer, output_buffer, wf_step_contents)


In [133]:
qual_portname[qual_portname['port_name'] =='max_preds']

,program_name,port_name,qualified_portname,port_type,data_id
4,calculate_Models,max_preds,max_preds,IN,13
8,defineLinearModels,max_preds,max_preds,IN,22
15,get_predictor_matrix,max_preds,get_predictor_matrix/max_preds,OUT,13


In [89]:
qual_portname[qual_portname['port_name'] =='max_preds']

,program_name,port_name,qualified_portname,port_type,data_id
4,calculate_Models,max_preds,max_preds,IN,13
8,defineLinearModels,max_preds,max_preds,IN,22
15,get_predictor_matrix,max_preds,get_predictor_matrix/max_preds,OUT,13
